In [1]:
import numpy as np 
from astropy.io import fits
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib ipympl

In [3]:
'''
column 1: group ID
column 2: richness
column 3: ra
column 4: dec
column 5: redshift
column 6: log M_h/(M_sun/h)
column 7: L_group (10^10Lsun/h/h)
'''

dgroup = np.loadtxt('../odata/CLAUDS_HSC_iband_group')
digal = np.loadtxt('../odata/CLAUDS_HSC_iband_igal')
d2 = np.loadtxt('../odata/iCLAUDS_HSC_iband_2')

In [144]:
np.where((dgroup[:,4] > 0 ) & (dgroup[:,4] < 1) & (dgroup[:,1] > 50))[0].shape

(4765,)

In [4]:
def cal_galdr(zbin):  
    from astropy.coordinates import SkyCoord
    from astropy.cosmology import FlatLambdaCDM
    from astropy import units as u
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
    
    rac = 150 #deg
    dec = 2 #deg

    dra = 1
    ddec = 1
    idx_Gal = np.where((np.abs(digal[:,1] - rac) < dra) & (np.abs(digal[:,2] - dec) < ddec) & 
                       (digal[:,3] > zbin[0]) & (digal[:,3] <= zbin[1]))[0]

    ra_Gal = digal[idx_Gal, 1]
    dec_Gal = digal[idx_Gal, 2]
    redz_Gal = digal[idx_Gal, 3]
    sep = (ra_Gal - 352)*u.deg
    d_A = cosmo.comoving_distance(z=redz_Gal) #Mpc
    d_r = (sep * d_A).to(u.Mpc, u.dimensionless_angles()) #to be a comoving distance
    
    return idx_Gal, d_r

In [5]:
def cal_rtheta(clu_ra,clu_dec,clu_redz):
    from astropy.coordinates import SkyCoord
    from astropy.cosmology import FlatLambdaCDM
    from astropy import units as u
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
    
    d_A = cosmo.comoving_distance(z=clu_redz)  #Mpc
    
    cclu = SkyCoord(clu_ra, clu_dec, unit="deg")
    cc = SkyCoord(150, 2.0, unit="deg")
#     sep = cc.separation(cclu)
    sep = (clu_ra - 150)*u.deg
    d_r = (sep * d_A).to(u.Mpc, u.dimensionless_angles()) #to be a comoving distance

    return d_r

In [6]:
def cal_zbin_Ng(zbin, Ng):
    rac = 150 #deg
    dec = 2.0 #deg

    dra = 1.0
    ddec = 1.0

    idx_Clu = np.where((np.abs(dgroup[:,2] - rac) < dra) & (np.abs(dgroup[:,3] - dec) < ddec) & 
                       (dgroup[:,4] > zbin[0]) & (dgroup[:,4] <= zbin[1])
                       & (dgroup[:,1] >= Ng))[0]
    ra_Clu = dgroup[idx_Clu, 2]
    dec_Clu = dgroup[idx_Clu, 3]
    redz_Clu = dgroup[idx_Clu, 4]
    mass_Clu = dgroup[idx_Clu, 5]

    d_r = cal_rtheta(ra_Clu,dec_Clu,redz_Clu)
    print(d_r.shape)
    return idx_Clu, d_r

In [19]:
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 1.2 #set the value globally

# plt.figure(figsize = (12,7), dpi = 500)
plt.figure(figsize = (12,3.5))

#===========================================================
#plot galaxy distribution
#===========================================================
# plt.subplot(211)
# zbin = [0.1, 1]
# idx_Gal, d_r = cal_galdr(zbin)
# redz_Gal = digal[idx_Gal, 3]
# d_r = d_r * 0.674 #Mpc/h
# for i in range(d_r.shape[0]):
#     limz = (redz_Gal[i]*6 + 1) #rlim = z + 5
#     if np.abs(d_r.value[i]) <= limz:
#         plt.plot(redz_Gal[i], d_r[i], '.', c = 'k', alpha= 0.6, mec = 'none', ms = 1) 

# plt.xlim(0.1,1)
# ax = plt.gca()
# ax.set_xticks([0.5,1,1.5,2,2.0,2.5,3,3.5,4,4.5,5])
# ax.set_xticks([0.1], minor = True)
# ax.set_xticklabels(['0.1'], minor = True)
        
# plt.xlim(0.1,5.0)
# plt.ylim(-14, 14)
# plt.xlabel('Redshift', fontsize = 12)
# plt.ylabel('Seperation [Mpc/$h]$', fontsize = 12)
# plt.tick_params(top = 'on', right = 'on', which='both',direction = 'in',labelsize = 11)
# plt.text(0.2, 10, 'Galaxies', color = 'k', fontsize = 12)

# ax2 = ax.twiny()
# ax2.set_xlim(ax.get_xlim())
# RedzD = [0.1742, 0.3666, 0.5837, 0.834, 1.1289, 1.4838, 1.9206, 2.4699, 3.1802, 4.1237]
# ax2.set_xticks(RedzD)
# ax2.set_xticklabels([500,1000,1500,2000,2500,3000,3500,4000,4500,5000])
# ax2.set_xticks([2.1783, 2.8012, 3.6168, 4.7173], minor = True)
# ax2.set_xticklabels([3750, 4250, 4750, 5250], minor = True)
# ax2.set_xlabel('Comoving distance [Mpc/$h]$', fontsize = 12)
# plt.tick_params(top = 'on', right = 'on', which='both',direction = 'in',labelsize = 11)

# plt.tight_layout()


# #===========================================================
# #plot group distribution
# #===========================================================
# plt.subplot(212)
import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 1.2 #set the value globally

import matplotlib.cm as cm
from matplotlib.colors import Normalize
cmap = cm.jet
norm = Normalize(vmin=2, vmax=7)

#0 < z < 2.5
zbin = [0,2.5]; Ng = 10;
idx_Clu, d_r = cal_zbin_Ng(zbin, Ng)
redz_Clu = dgroup[idx_Clu, 4]
mass_Clu = dgroup[idx_Clu, 5]
d_r = d_r * 0.674
for i in range(d_r.shape[0]):
    limz = (redz_Clu[i]*1.5 + 5) #rlim = z + 5
    if np.abs(d_r.value[i]) <= limz:
        plt.plot(redz_Clu[i], d_r[i], 'o', c = 'grey', ms = 1.1*(mass_Clu[i] - 10), mec = 'none', alpha = 0.6)     
plt.text(0.5, -11, r'$0.1 < z \leq 2.5:\ N_{\rm g} \geq 10$', color = 'grey', fontsize = 13)


#2 < z < 6
zbin = [2.5,6]; Ng = 2;
idx_Clu, d_r = cal_zbin_Ng(zbin, Ng)
redz_Clu = dgroup[idx_Clu, 4]
mass_Clu = dgroup[idx_Clu, 5]
Ng_Clu = dgroup[idx_Clu, 1]
d_r = d_r * 0.674
for i in range(d_r.shape[0]):
    limz = redz_Clu[i]*1.5 + 5 #rlim = z + 5
    if np.abs(d_r.value[i]) <= limz:
        plt.plot(redz_Clu[i], d_r[i], 'o', c = cmap(norm(Ng_Clu[i])), ms = 1.1*(mass_Clu[i] - 10), mec = 'none', alpha = 0.8)
plt.text(3.5, -11, r'$2.5 < z \leq 5:\ N_{\rm g} \geq 2$', color = 'b', fontsize = 13)

ax = plt.gca()

ax.set_xticks([0.5,1,1.5,2,2.0,2.5,3,3.5,4,4.5,5])
ax.set_xticks([0.1], minor = True)
ax.set_xticklabels(['0.1'], minor = True)
# ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
plt.xlabel('Redshift', fontsize = 12)
plt.ylabel('Separation [$h^{-1}$Mpc]', fontsize = 14)
plt.xlim(0.1,5)
plt.ylim(-14,14)
plt.tick_params(top = 'on', right = 'on', which='both',direction = 'in',labelsize = 12)
# plt.text(0.2, 10, 'Groups and Protocluster candidates', color = 'k', fontsize = 12)

ax2 = ax.twiny()
ax2.set_xlim(ax.get_xlim())
RedzD = [0.1742, 0.3666, 0.5837, 0.834, 1.1289, 1.4838, 1.9206, 2.4699, 3.1802, 4.1237]
ax2.set_xticks(RedzD)
ax2.set_xticklabels([500,1000,1500,2000,2500,3000,3500,4000,4500,5000])

ax2.set_xticks([2.1783, 2.8012, 3.6168, 4.7173], minor = True)
ax2.set_xticklabels([3750, 4250, 4750, 5250], minor = True)
ax2.set_xlabel('Comoving distance [$h^{-1}$Mpc]', fontsize = 14)
plt.tick_params(top = 'on', right = 'on', which='both',direction = 'in',labelsize = 12)

cax = plt.axes([0.94, 0.16, 0.012, 0.673])
cbar = plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax = cax, pad = 0.007)
cbar.set_label(r'$N_{\rm g}$', fontsize = 13)
cbar.ax.tick_params(labelsize=12) 

plt.tight_layout(rect = (0, 0, 0.95, 1))

plt.savefig('./z-map.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(11619,)
(8221,)


/opt/anaconda-mamba/envs/mamba-python3/lib/python3.7/site-packages/ipykernel_launcher.py:115: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [7]:
def drive_comovD_from_z(comovD):
    from astropy.cosmology import FlatLambdaCDM
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
    
    zinitial = 2.01
    Dini = cosmo.comoving_distance(zinitial).value
    DeltaD = comovD - Dini
    while np.abs(DeltaD) > 0.5:
        if DeltaD > 0:
            zinitial += 0.00001
        else:
            zinitial -= 0.00001
        Dini = cosmo.comoving_distance(zinitial).value
        DeltaD = comovD - Dini
    return zinitial

In [55]:
# Dv = np.array([500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500]) / 0.674
Dv = np.array([3750, 4250, 4750, 5250]) / 0.674
z0 = []
for i in range(Dv.shape[0]):
    z0.extend([drive_comovD_from_z(Dv[i])])

In [56]:
RedzD = np.round(z0, 4)
print(RedzD)
cosmo.comoving_distance(RedzD)

[2.1783 2.8012 3.6168 4.7173]


<Quantity [5563.33798978, 6305.12078077, 7046.99337048, 7788.80675429] Mpc>

In [229]:
drive_comovD_from_z(7800)

4.735899999985177

In [44]:
from astropy import units as u
from astropy.coordinates import SkyCoord
c1 = SkyCoord(ra = 150, unit="deg")
c2 = SkyCoord(ra = 151, unit="deg")
c1.separation(c2)

TypeError: ICRS() missing 1 required positional argument: 'dec' (or first argument should be an instance of UnitSphericalRepresentation).

## test region size

In [10]:
def cal_rtheta0(clu_ra,clu_dec,clu_redz):
    from astropy.coordinates import SkyCoord
    from astropy.cosmology import FlatLambdaCDM
    from astropy import units as u
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
    
    d_A = cosmo.comoving_distance(z=clu_redz) * 0.674
    
    cclu = SkyCoord(clu_ra, clu_dec, unit="deg")
    cc = SkyCoord(150, 2.0, unit="deg")
    sep = cc.separation(cclu)
    sep = (clu_ra - 150)*u.deg
    d_r = (sep * d_A).to(u.Mpc, u.dimensionless_angles()) #to be a comoving distance

    return d_r

In [11]:
#COSMOS galaxies
ztest = 0.05
idx_gal = np.where((digal[:,1] > 148) & (digal[:,1] < 152) &
                  (digal[:,2] > 0.5) & (digal[:,2] < 4.0) & 
                  (np.abs(ztest - digal[:,3]) < 0.2))[0]
ra_gal = digal[idx_gal, 1]
dec_gal = digal[idx_gal, 2]
redz_gal = digal[idx_gal, 3]

d_r = cal_rtheta0(ra_gal, dec_gal, redz_gal) 
plt.figure()

plt.scatter(ra_gal, dec_gal, c = np.abs(d_r), cmap = 'jet', s = 0.5)
plt.colorbar()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [327]:
#COSMOS cluster
ztest = 0.05
idx_grp = np.where((dgroup[:,2] > 148) & (dgroup[:,2] < 152) &
                  (dgroup[:,3] > 0.5) & (dgroup[:,3] < 4.0) & 
                  (np.abs(ztest - dgroup[:,4]) < 0.01))[0]
ra_grp = dgroup[idx_grp, 2]
dec_grp = dgroup[idx_grp, 3]
redz_grp = dgroup[idx_grp, 4]

d_r = cal_rtheta0(ra_grp, dec_grp, redz_grp) 
plt.figure()

plt.scatter(ra_grp, dec_grp, c = np.abs(d_r), cmap = 'jet', s = 0.5)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
zbin = [0.1,1]

rac = 150 #deg
dec = 2 #deg

dra = 1
ddec = 1
idx_Gal = np.where((np.abs(digal[:,1] - rac) < dra) & (np.abs(digal[:,2] - dec) < ddec) & 
                   (digal[:,3] > zbin[0]) & (digal[:,3] <= zbin[1]) & 
                  (digal[:,4] < 26))[0]

redz_Gal = digal[idx_Gal, 3]

plt.figure()
plt.hist(redz_Gal, bins = 40)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 3783.,  5959.,  7989.,  8039.,  7502.,  7477.,  6226., 10081.,
         7631., 16043., 10661.,  7348.,  8461., 13082., 11446., 10784.,
         6441.,  6928.,  8902., 10482.,  9920.,  8695.,  8870.,  8638.,
        11221., 10254.,  8376., 14313., 17493., 21191., 17089., 11355.,
        15177., 10736.,  7397.,  7585., 10912., 14502., 11280., 10692.]),
 array([0.1001   , 0.1225975, 0.145095 , 0.1675925, 0.19009  , 0.2125875,
        0.235085 , 0.2575825, 0.28008  , 0.3025775, 0.325075 , 0.3475725,
        0.37007  , 0.3925675, 0.415065 , 0.4375625, 0.46006  , 0.4825575,
        0.505055 , 0.5275525, 0.55005  , 0.5725475, 0.595045 , 0.6175425,
        0.64004  , 0.6625375, 0.685035 , 0.7075325, 0.73003  , 0.7525275,
        0.775025 , 0.7975225, 0.82002  , 0.8425175, 0.865015 , 0.8875125,
        0.91001  , 0.9325075, 0.955005 , 0.9775025, 1.       ]),
 <BarContainer object of 40 artists>)